In [1]:
import random

In [2]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        The move is valid if:
        - The selected pit number is in range (1-indexed by player perspective)
        - The selected pit contains at least one stone
        """
        pit -= 1
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + pit
            if pit_index < self.p1_pits_index[0] or pit_index > self.p1_pits_index[1]:
                return False
        else:
            pit_index = self.p2_pits_index[0] + pit
            if pit_index < self.p2_pits_index[0] or pit_index > self.p2_pits_index[1]:
                return False
        return self.board[pit_index] > 0

        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        random.seed(109)
        if self.current_player == 1:
            valid_pits = [i - self.p1_pits_index[0] + 1 
                        for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1) 
                        if self.board[i] > 0]
        else:
            valid_pits = [i - self.p2_pits_index[0] + 1
                        for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)
                        if self.board[i] > 0]

        if not valid_pits:
            return None

        return random.choice(valid_pits)
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit.
        It performs the following:
        1. Validates the chosen pit (1-indexed from player's perspective).
        2. Checks if the game has ended.
        3. Distributes stones counter-clockwise, skipping opponent's mancala.
        4. Applies capture rule if last stone lands in an empty pit on player's side.
        5. Records the move.
        6. Switches to the other player's turn (no extra turns).
        """
        if not self.valid_move(pit):
            print("INVALID MOVE")
            return self.board
        if self.winning_eval():
            print("GAME OVER")
            return self.board
            
        print(f"Player {self.current_player} chose pit: {pit}")
        pit -= 1
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + pit
        else:
            pit_index = self.p2_pits_index[0] + pit

        stones = self.board[pit_index]
        self.board[pit_index] = 0
        index = pit_index
        while stones > 0:
            index = (index + 1) % len(self.board)
            if self.current_player == 1 and index == self.p2_mancala_index:
                continue
            if self.current_player == 2 and index == self.p1_mancala_index:
                continue
            self.board[index] += 1
            stones -= 1

        if self.current_player == 1 and self.p1_pits_index[0] <= index <= self.p1_pits_index[1] and self.board[index] == 1:
            opposite = self.p2_pits_index[1] - (index - self.p1_pits_index[0])
            self.board[self.p1_mancala_index] += self.board[opposite] + 1
            self.board[opposite] = 0
            self.board[index] = 0
        elif self.current_player == 2 and self.p2_pits_index[0] <= index <= self.p2_pits_index[1] and self.board[index] == 1:
            opposite = self.p1_pits_index[1] - (index - self.p2_pits_index[0])
            self.board[self.p2_mancala_index] += self.board[opposite] + 1
            self.board[opposite] = 0
            self.board[index] = 0

        self.moves.append((self.current_player, pit + 1))
        self.current_player = 2 if self.current_player == 1 else 1
        return self.board

    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_empty = all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1))
        p2_empty = all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1))
        if not (p1_empty or p2_empty):
            return False
        if not p1_empty:
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
                self.board[self.p1_mancala_index] += self.board[i]
                self.board[i] = 0
        if not p2_empty:
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
                self.board[self.p2_mancala_index] += self.board[i]
                self.board[i] = 0
        return True